In [1]:
!pip install logicalinduction

In [2]:
import logicalinduction as li

In [3]:
li.find_credences?

In [6]:
sentence1 = li.Atom("socrates is a man")

In [8]:
sentence2 = li.Atom("socrates is mortal")

In [11]:
sentence3 = li.Implication(sentence1, sentence2)

In [12]:
sentence3

socrates is a man → socrates is mortal

In [24]:
def trade_on_probability(sentence, day, p, slope=10):
    return li.Min(
        li.Constant(1),
        li.Max(
            li.Constant(-1),
            li.Sum(
                li.Constant(slope * p),
                li.Product(
                    li.Constant(-slope),
                    li.Price(sentence1, day)
                )
            )
        )
    )

In [25]:
def trading_alg(probability):
    for i in li.integers(start=1):
        yield {sentence1: trade_on_probability(sentence1, i, probability)}

In [30]:
belief_history = li.History([{sentence1: .1}])

In [31]:
belief_history.lookup(sentence1, 1)

0.1

In [26]:
first_trading_policy = next(trading_alg(.8))

# Here we have a title